# PATH CONFIG

In [35]:
import os

print(os.getcwd())
if not os.getcwd().endswith("app"):
    os.chdir("../app")
    print(os.getcwd())

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

%load_ext autoreload
%autoreload 2
# %matplotlib inline

/home/turbotowerlnx/Documents/Master/TA/TA-Spanish-Esperanto-Translator/app
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import pandas as pd
from dataclasses import dataclass

@dataclass
class Configuration:
    DATA_PATH: str = os.path.join("..", "data")
    RAW_DATA_PATH: str = os.path.join(DATA_PATH, "raw")
    PROCESSED_DATA_PATH: str = os.path.join(DATA_PATH, "processed")
    MODELS_PATH: str = os.path.join("..", "models")
    RESULTS_PATH: str = os.path.join("..", "results")

    spanish_data: str = os.path.join(RAW_DATA_PATH, "CCMatrix.eo-es.es")
    esperanto_data: str = os.path.join(RAW_DATA_PATH, "CCMatrix.eo-es.eo")

    corpus_path: str = os.path.join(PROCESSED_DATA_PATH, "corpus.csv")
    corpus_raw_path: str = os.path.join(PROCESSED_DATA_PATH, "corpus_raw.csv")


CONFIG = Configuration()

# EXPERIMENTS

In [37]:
def read_moses_bitext(source_path, target_path):
    """
    Reads a Moses-formatted parallel corpus from two aligned files.
    
    :param source_path: Path to the source language file (e.g., 'corpus.es')
    :param target_path: Path to the target language file (e.g., 'corpus.eo')
    :return: A list of (source_sentence, target_sentence) tuples
    """
    corpus = []
    
    with open(source_path, 'r', encoding='utf-8') as src_file, \
         open(target_path, 'r', encoding='utf-8') as trg_file:
        
        for src_line, trg_line in zip(src_file, trg_file):
            source_sentence = src_line.strip()
            target_sentence = trg_line.strip()
            
            if source_sentence and target_sentence:
                corpus.append((source_sentence, target_sentence))
    
    return corpus

In [38]:
corpus = read_moses_bitext(CONFIG.spanish_data, CONFIG.esperanto_data)

In [39]:
print(f"{len(corpus):_}")

5_750_168


In [40]:
df_corpus = pd.DataFrame(corpus, columns=["Spanish", "Esperanto"])
df_corpus.head()

,Spanish,Esperanto
0,"Se miran unos a otros, interrogándose con la m...",Ili rigardas unu la alia[n].
1,"""Sí, ¿pero en Brasil no mentirán sobre algo así?","""Jes, sed en Brazilo ili ne mensogos pri io si..."
2,"Has estado en AT5, ellos saben todo ...","Vi estis ĉe AT5, ili scias ĉion ..."
3,Dijeron el Viento y el Bosque: «¡Ninguno de lo...,"* [[1909]]: [[Elżbieta Zawacka]], pola [[kurie..."
4,"Unos a otros se miran,",Ili rigardas unu la alia[n].


In [41]:
df_corpus.sample(10)


,Spanish,Esperanto
2563809,"Es una historia muy entretenida de misterio, a...","Ĝi temas pri historio plena de agado, romantik..."
5085917,(→Artículos publicados),(redaktu Diskutindajn artikolojn)
26154,"En efecto, Él mismo es la Verdad (Cf.","Fakte, Li mem estas la Vero (kp."
3125527,Amábamos la muchacha rubia,Al mi plaĉas blond-knabino
598597,ITA2 eran a su vez basado en el código telegrá...,ITA2 estis en victurno surbaze de la 5-bita te...
569718,Milankovitch ritmo parece explicar mejor los e...,Milankovitch kiun paŝadado ŝajnas venki klarig...
2035957,"Es también su concentración, su capacidad de i...","Ĝi ankaŭ estas ilia koncentriĝo, ilia kapacito..."
4019088,¡Que bueno es saber que Dios está con nosotros!,Kiel feliĉe estas scii ke Dio ĉiopava kaj ĉios...
4566568,La vida en Murasame no es fácil.,La laboroj en Mumbajo ne estas tiel facile tro...
1726316,2004 Memoria de España TVE -,2004 Memoro de Hispanio TVE


In [42]:
df_corpus.to_csv(CONFIG.corpus_raw_path, index=False)

# Cleaning

### Remove weird patterns

In [43]:
wiki_pattern = r'\[\['
rows_to_drop = df_corpus['Esperanto'].str.contains(wiki_pattern, na=False)
df_corpus_cleaned = df_corpus[~rows_to_drop].copy()

print(f"Original samples: {len(df_corpus):_}")
print(f"Cleaned samples: {len(df_corpus_cleaned):_}")
print(f"Dropped samples: {len(df_corpus) - len(df_corpus_cleaned):_}")

Original samples: 5_750_168
Cleaned samples: 5_741_814
Dropped samples: 8_354


### Lowercase

In [44]:
df_corpus_cleaned = df_corpus_cleaned.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
df_corpus_cleaned.head()

,Spanish,Esperanto
0,"se miran unos a otros, interrogándose con la m...",ili rigardas unu la alia[n].
1,"""sí, ¿pero en brasil no mentirán sobre algo así?","""jes, sed en brazilo ili ne mensogos pri io si..."
2,"has estado en at5, ellos saben todo ...","vi estis ĉe at5, ili scias ĉion ..."
4,"unos a otros se miran,",ili rigardas unu la alia[n].
5,tiene un significado semejante al de gehena.,"tiel ""geheno"" (minuskle) fariĝis (nuntempe eks..."


In [45]:
df_corpus_cleaned.to_csv(CONFIG.corpus_path, index=False)